In [2]:
import numpy as np

## Gradient Descent: Optimizing U-V Decomposition w RMSE

In [216]:
utility = np.array([[5, 2, 4, 4, 3], [3, 1, 2, 4, 1], [2, 0, 3, 1, 4], [2, 5, 4, 3, 5], [4, 4, 5, 4, 0]])

In [8]:
print(utility)

[[5 2 4 4 3]
 [3 1 2 4 1]
 [2 0 3 1 4]
 [2 5 4 3 5]
 [4 4 5 4 0]]


In [14]:
starting_u = np.ones((5, 2))
starting_v = np.ones((2, 5))

In [27]:
print(starting_u, '<-- U', '\n')
print(starting_v, '<-- V' '\n')
print(np.matmul(starting_u, starting_v), '<-- U x V')

[[1. 1.]
 [1. 1.]
 [1. 1.]
 [1. 1.]
 [1. 1.]] <-- U 

[[1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1.]] <-- V

[[2. 2. 2. 2. 2.]
 [2. 2. 2. 2. 2.]
 [2. 2. 2. 2. 2.]
 [2. 2. 2. 2. 2.]
 [2. 2. 2. 2. 2.]] <-- U x V


In [38]:
one = np.matmul(starting_u, starting_v)

In [95]:
def rmse(matrix1, matrix2):
    zipped = zip(matrix1.reshape(-1), matrix2.reshape(-1)) 
    nonzero = list(filter(lambda x: x[0] != 0 or x[1] != 0, zipped)) # our RMSE is only calculated over rated items
    squared_errors = [(x[0]-x[1])**2 for x in nonzero]
    return (sum(squared_errors)/len(squared_errors))**0.5

In [129]:
def get_changing_vector(utility, starting_u, starting_v, optimizing):
    mat = np.matmul(starting_u, starting_v)
    if optimizing[0] == 0:
        # change row number optimizing[1]
        arbitrary = mat[optimizing[1]]
        u = utility[optimizing[1]]
    else:
        # change column number optimizing[2]
        arbitrary = mat[:, optimizing[2]]
        u = utility[:, optimizing[2]]
    return arbitrary, u
    

In [179]:
get_changing_vector(utility, starting_u, starting_v, (0, 0, 0))

(array([2., 2., 2., 2., 2.]), array([5, 0, 4, 4, 3]))

In [232]:
from sympy.solvers import solve
from sympy import *

def optimize(utility, starting_u, starting_v, optimizing):
    arbitrary, u = get_changing_vector(utility, starting_u, starting_v, optimizing)
    x = Symbol('x')
    row = (x, starting_u[optimizing[1], optimizing[2]])
    col = [starting_v[:,i] for i in range(starting_v.shape[1])]
    eq = [np.dot(row, b) for b in col]
    eq2 = [(i[0]-i[1])**2 for i in zip(u, eq)]
    eq3 = sum(eq2)
    return solve(diff(eq3, x), x)[0]

    

In [233]:
optimize(utility, starting_u, starting_v, (0, 0, 0))

2.60000000000000

In [132]:
from sympy.solvers import solve
from sympy import Symbol



In [137]:
sum(eq), eq

(6*x + 6, [x + 1, 2*x + 2, 3*x + 3])

In [245]:
def optimal_value(utility, starting_u, starting_v, optimizing):
    # choice = (0, 0, 0) means first element of first matrix
    mat = np.matmul(starting_u, starting_v)
    arbitrary, u = get_changing_vector(utility, starting_u, starting_v, optimizing)
    print('Starting RMSE of Changing Values: ', rmse(arbitrary, u))
    print('Starting Total RMSE: ', rmse(mat, utility))
    
    new_value = optimize(utility, starting_u, starting_v, optimizing)
    print('Optimal Value: ', new_value)
    starting_u[optimizing[1], optimizing[2]] = new_value
    new_mat = np.matmul(starting_u, starting_v)
    print(new_mat, '<-- updated matrix')

    print('New Total RMSE: ', rmse(new_mat, utility))
    
    

In [246]:
print(optimal_value(utility, starting_u, starting_v, (0, 0, 0)))

Starting RMSE of Changing Values:  1.8973665961010275
Starting Total RMSE:  1.8220867158288598
Optimal Value:  2.60000000000000
[[3.6 3.6 3.6 3.6 3.6]
 [2.  2.  2.  2.  2. ]
 [2.  2.  2.  2.  2. ]
 [2.  2.  2.  2.  2. ]
 [2.  2.  2.  2.  2. ]] <-- updated matrix
New Total RMSE:  1.675708805252273
None


In [98]:
utility

array([[5, 0, 4, 4, 3],
       [3, 0, 2, 4, 1],
       [2, 0, 3, 1, 4],
       [2, 0, 4, 3, 5],
       [4, 0, 5, 4, 0]])

In [99]:
mat = np.matmul(starting_u, starting_v)

In [101]:
rmse(utility, mat)

1.8439088914585775

In [75]:
utility[:, 1]

array([2, 1, 0, 5, 4])

In [74]:
utility[0, 1]

2

In [77]:
changed = utility
changed

array([[5, 2, 4, 4, 3],
       [3, 1, 2, 4, 1],
       [2, 0, 3, 1, 4],
       [2, 5, 4, 3, 5],
       [4, 4, 5, 4, 0]])

In [78]:
changed[:, 1] = [0, 0, 0, 0, 0]

In [79]:
changed

array([[5, 0, 4, 4, 3],
       [3, 0, 2, 4, 1],
       [2, 0, 3, 1, 4],
       [2, 0, 4, 3, 5],
       [4, 0, 5, 4, 0]])

# Resources
- The structure of this is based on Chapter 9 of **Mining of Massive Datasets**: http://infolab.stanford.edu/~ullman/mmds/book.pdf
- Libraries for recommendation systems: 
    - Surprise: https://surprise.readthedocs.io/en/stable/index.html
    - LightFM: https://lyst.github.io/lightfm/docs/index.html